In [3]:
import numpy as np
import pandas as pd
import polars as pl

# See readme for how I obtained the the below data
#catalog = pd.read_csv('../data/1680058493237O-result.csv', sep=',', header=0, dtype=np.float64)

# smaller sampe of the above data for testing purposes
catalog = pd.read_csv('../data/test.csv', sep=',', header=0, dtype=np.float64)

# arbitrary threashold 'k is for Kalle'
threshold_k = np.pi / 4

# helper functio
deg2rad = lambda rad: rad * np.pi / 180

# sort the catalog by magnitudes
stars = catalog.sort_values(by='b1mag')

stars

stars['dec']

0    -16.716001
1    -52.695661
2    -60.835273
3     19.182450
4     38.783690
        ...    
53    71.612316
54    71.616219
55    66.635347
45    75.033424
82    73.264174
Name: dec, Length: 83, dtype: float64

In [2]:
# This bit of math is a common operation in astronomy and is used to convert the spherical coordinates (r, \theta, \phi) to cartesian coordinates (x, y, z)
# To be more specific, if (RA, Dec) represents the spherical coordinates of a celestial object, then we can convert them to Cartesian coordinates (x, y, z) 
# using the following formulas:
#   x = cos(dec) * cos(ra)
#   y = cos(dec) * sin(ra)
#   z = sin(dec)
#
rays = np.stack(
            (   np.cos(deg2rad(stars['dec'])) * np.cos(deg2rad(stars['ra'])), 
                np.cos(deg2rad(stars['dec'])) * np.sin(deg2rad(stars['ra'])), 
                np.sin(deg2rad(stars['dec']))
            ) 
        ).T
rays

# Conversely, if we have the Cartesian coordinates (x, y, z), we can convert them to spherical coordinates (r, θ, φ) using the following formulas:
#
# r = sqrt(x^2 + y^2 + z^2)
# θ = arccos(z/r)
# φ = atan2(y, x)


array([[-0.18745319,  0.93921852, -0.287628  ],
       [-0.06322262,  0.60274195, -0.79542759],
       [-0.37385775, -0.31259124, -0.87322225],
       [-0.78378669, -0.52698704,  0.32857736],
       [ 0.12509649, -0.76941311,  0.62638194],
       [ 0.19505172,  0.97036265, -0.14265749],
       [ 0.13050036,  0.68231601,  0.71931531],
       [ 0.02088846,  0.99143518,  0.12891845],
       [-0.41811213,  0.9038191 ,  0.09106749],
       [ 0.49272291,  0.22380426, -0.84091366],
       [-0.42393818, -0.2542838 , -0.86926185],
       [ 0.55971537, -0.82768132, -0.04077179],
       [ 0.34390209,  0.89497374,  0.28417136],
       [ 0.45922089, -0.87484256,  0.15416441],
       [-0.34481839, -0.82641108, -0.44513482],
       [-0.91408008, -0.3563536 , -0.19357096],
       [-0.39151333,  0.79116116,  0.46987374],
       [ 0.83733252, -0.23358762, -0.49427833],
       [ 0.45564906, -0.53618207,  0.71055804],
       [-0.44940405, -0.0523942 , -0.89179081],
       [-0.49379562, -0.10433183, -0.863

Now we have and array of tuples with precalulated data for each star in the catalog.  Magnitude is ignored. 

In [47]:
numrows = rays.shape[0]

rays[0,:]  # get the row at position 0 of the rays collection
rays[0+1,:] # gets the next row of the rays collection  

rays[0,:] * rays[0+1:,:] # gets the next row of the rays collection  
(rays[0,:] * rays[0+1:,:]).sum(-1) # gets the next row of the rays collection  

rays[0+1:,:] # this is where I loose track?  
d = np.arccos((rays[0,:] * rays[0+1:,:]).sum(-1) )

flat = np.flatnonzero(d < threshold_k)
flat


array([ 0,  4,  6,  7, 21, 26, 28, 32])

In [29]:
i_, j_, d_ = [],[],[]
for i in range(0, numrows):
    d = np.arccos((rays[i,:] * rays[i+1:,:]).sum(-1) )
    for j in np.flatnonzero(d < threshold_k):
        i_.append(i)
        j_.append(i+1+j)
        d_.append(d[j])

pairs = pd.DataFrame({'i':np.array(i_), 'j':np.array(j_), 'd':np.array(d_)}).sort_values(by='d')

pairs


,i,j,d
1190,61,65,0.007804
578,40,77,0.021592
1126,58,64,0.025261
1075,56,60,0.032253
1198,61,73,0.037113
...,...,...,...
210,31,41,0.779367
1083,56,68,0.779936
110,16,49,0.781039
989,52,80,0.782674
